<a href="https://colab.research.google.com/github/johanhoffman/DD2363-VT19/blob/bozzato/Lab-2/bozzato_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Direct methods**
**Bozzato Federico**

# **Abstract**

In this lab we will see how these general methods can be used to solve algebraic problems, such as to find the inverse of a matrix or to compute the eigenvalues of a certain matrix, which are theoretically simple to solve but very hard to compute.

#**About the code**

**Author:** Federico Bozzato 

In [1]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2019 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [0]:
# Load neccessary modules.
from google.colab import files

import sys
import math
import numpy as np

# **Introduction**

Direct methods are general methods for constructing a proof of the existence of a minimizer for a given functional [1]. This kind of methods are widely used to solve several algebraic problems, which may be computationally difficult to implement: for example to find eigenvalues and eigenvectors of an $n\times n$ matrix with $n \ge 4$ is a non trivial problem to solve.

Even though some of these methods can give us approximate results, for particular types of inputs these methods reach high performances. In fact, in addition to being used to prove the existence of a solution,  direct methods may be used to compute the solution to derised accurancy [1].


This report is divided into three parts:

1. Methods: in this section, each algorithm is presented and explained, giving also the mathematic definition of the operation the algorithm implements

2. Results: in this section, the results of each algorithm are presented.

3. Discussion: in this last section, results are discussed.

# **Methods**

This section is divided into two subsection: the mandatory part and the extra assignment.

## Mandatory assignment

###1. QR factorization
####Definition
QR factorization is a method used to solve a generic equation such as $Ax=b$ where $A$ is an $n\times n$ matrix, $b$ is an $n\times 1$ vector and $x$ is the unknown vector. 

The key idea of QR factorization consists of decomposing an $n\times n$ matrix $A$ into the product $A=QR$, where $Q$ is an orthogonal matrix (which means $Q^{-1}=Q^{T}$) and $R$ is an upper triangular matrix [2].

####Implementation
One way to implement QR factorization is to start from Gram-Schmidt algorithm, that orthogonalizes vectors belonging to the same vector space $V$. 

Given a square matrix $A\in\mathbb{R}^{n\times n}$, with its columns defining the vectors $\{a_{i}\}_{i=1}^n$, the new orthogonal vectors are given by [3]

>$
v_j = a_j - \displaystyle \sum_{i=1}^{j-1} \left(a_j,q_i\right)q_i \qquad \forall j=1,\dots,n
$

where

* $a_j$ is the $j-$th column of matrix $A$
* $q_i$ is the normalized vector $q_j= \frac{v_i}{||v_i||}$

These equations can be rewritten in the following way

>$
  \begin{align*}
    a_1 &= r_{11}q_1 \\
    a_2 &= r_{12}q_1 + r_{22}q_2 \\
           &\vdots\\
    a_n &= r_{1n}q_1 + \dots + r_{nn}q_n
  \end{align*}
$

where

* $r_{ij}=(a_j,q_i)$
* $r_{jj}=||a_j-\sum_{i=1}^{j-1}(a_j,q_i)q_i||$

As it is possible to observe, the rewritten equations corresponds to product $A=QR$, where the columns of $Q$ are given by ${q_i}_{i=1}^n$ and $R$ is an uppen triangular matrix.

The algorithm implementing the QR factorization is the following one [4]:


```
for i = 1 to n do
  v_i = a_i
  r_ii = abs(v_i)
  q_i= v_i / r_ii
  for j = i+1 to n do
    r_ij = transpose(q_i)v_j
    v_j = v_i - r_ij*q_i
  end
end
```



In [0]:
def QRfactorization(matrixA):
	'''
	Calculates the QR factorization of matrixA which must be invertible
	Parameters:
	- matrixA: square matrix to be factorized and returns the orthogonal matrix Q and 
	the upper triangular matrix R such that A= QR
	Output:
	- Q: orthogonal matrix
	- R: upper triangular matrix 
	'''
	if not isinstance(matrixA,np.ndarray):
		matrixA= np.array(matrixA)
	
	
	if matrix_rank(matrixA) != matrixA.shape[1]:
		print('Error: matrix determinat is equal to 0. Impossible to do the',
			'factorization!')
		sys.exit(1)	
		
	if matrixA.shape[0] != matrixA.shape[1]:
		print('Only square matrices are allowed!')
		sys.exit(1)
	
	n= matrixA.shape[0]
	
	v= np.zeros((n,n))
	for i in range(0,n):
		v[:,i]= matrixA[:,i]

	Q= np.zeros((n,n))
	R= np.zeros((n,n))
	for i in range(0,n):
		r_ii= np.dot(v[:,i],v[:,i])**(0.5)
		R[i,i]= r_ii
		q_i= v[:,i]/r_ii
		Q[:,i]= q_i
		
		for j in range(i+1,n):
			r_ij= np.dot(q_i,v[:,j])
			R[i,j]= r_ij 
			v[:,j] = v[:,j] - r_ij*q_i
			
	return Q,R


###2. Direct solver $Ax=b$
####Definition
Let $A$ and $b$ be an $n\times n$ matrix and $n\times 1$ vector, respectively. It is possible to solve the system of equations $Ax=b$ by using the QR factorization. In fact, $A$ can be decomposed into the product $A=QR$, where $Q$ is a orthogonal matrix and $R$ is an upper triangular matrix. Thus,

>$
  Ax= B \Longrightarrow QRx=b
$

Since $Q^{-1}= Q^{T}$ because $Q$ is orthogonal, we obtain

>$
Rx= Q^{T}b
$

Now, in order to get $x$, we can use the *backward substitution* explained in [5].


####Implementation
One of the possible implementations is the following one:

In [0]:
def linearSystemSolver(matrixA,b):
	'''
	Calculates the unknown vector x of the equation Ax=b
	''' 
	
	# Ax=b -> QRx=b -> Rx=Q.Tb -> backward substitution
	Q, R = QRfactorization(matrixA);
	
	if not isinstance(matrixA,np.ndarray):
		matrixA= np.array(matrixA)
		
	if not isinstance(b,np.ndarray):
		b= np.array(b)
	
	
	new_b= np.dot(Q.T,b)
	
	x= np.zeros((matrixA.shape[1],1))
	x[-1]= new_b[-1]/R[-1][-1]
	for i in range(matrixA.shape[0]-2,-1,-1):
		s= 0
		for j in range(i+1,matrixA.shape[1]):
			s += R[i,j]*x[j]
		x[i]= (new_b[i] - s)/R[i,i]
	
	x= x.tolist()
	x= [x[i][0] for i in range(0,len(x))]
	
	return x

## Extra assignment

###3. Least squares problem $Ax=b$
####Definition
In the section about the direct solver we explored the algorithm for solving systems of linear equations $Ax=b$ where $A$ is a square $n\times n$ matrix. Now, let us consider a system of linear equations $Ax=b$ where, instead, $A$ is a $m\times n$ matrix, with $m>n$ and $b\in\mathbb{R}^m$. Since there exists no inverse matrix $A^{-1}$, we cannot use the previous algorithm to solve $Ax=b$ but we have to apply other algorithms.

We will examine the general case, where $b\notin \text{range}(A)$: for these cases, we say that the system $Ax=b$ is *overdetermined* [6]. Since there exists no exact solution $x$ to the system $Ax=b$, we have to find an approximation of the unknown vector $x$ that solves the system, and one method is called *least squares problem*. 

Least squares problem consists of minimizing the residual $r= b-Ax \in\mathbb{R}^m$, or similarly

>$
\displaystyle x=\arg \min_{y\in\mathbb{R}^n} ||b-Ay||
$

Since $b\notin\text{range}(A)$, looking for $x$ such that minimizes the residual $r=b-Ax$ means that we are seeking a vector $x\in\mathbb{R}^n$ such that the Euclidian distance betweem $Ax$ and $b$ is minimal [6]: this is equal to

>$
A^Tr= 0 \Longleftrightarrow A^TAx=A^Tb
$

And thus, $A^TA$ is always symmetric positive-semidefinite matrix because it has only real entries [7]. If $A^TA$ is invertible, the vector $x$ is given by

>$
  x= \left(A^TA\right)^{-1}A^Tb
$

####Implementation
For the implementation of the least squares problem, we start considering the equation

>$
  A^TAx= A^Tb
$

where

* $D=A^TA\in\mathbb{R}^{n\times n}$
* $x\in\mathbb{R}^n$
* $e=A^Tb\in\mathbb{R}^n$

Consequently, we can rewrite the equation $A^TAx= A^Tb$ as

>$
Dx=e
$

and use the direct solver algorithm presented in the previous section.

In [0]:
def leastSquareProblem(matrixA,b):
	if not isinstance(matrixA,np.ndarray):
		matrixA= np.array(matrixA)
	
	if matrixA.shape[0] < matrixA.shape[1]:
		print('Too many unknowns: impossible to solve the system')
		sys.exit(1)
		
	# problem: from Ax=b to x=(A.tA)^-1A.tb
	# first step: A.tAx= A.tb and then use linearSystemSolver
	
	A= np.dot(matrixA.T,matrixA)
	new_b= np.dot(matrixA.T,b)
	
	x= linearSystemSolver(A,new_b)
		
	return x 

###4. QR eigenvalue algorithm
####Definition
The eigenvalues for a matrix $A\in\mathbb{R}^{n\times n}$ are defined as the solution of the equation

>$
\det(A-\lambda I)=0
$

which is known as *characteristic equation* for $A$.

The eigenvectors $\{x_i\}$ associated to the $i-$th eigenvalue $\lambda_i$ are defined by 

>$
  \left(A-\lambda_iI\right)x=0
$


####Implementation
One way to seek the eigenvalues and their corresponding eigenvectors for a square $n\times n$ matrix $A$ is to find a matrix $T$ which is similar to $A$ by using similarity transforms. In fact, similar matrices have the same eigenvalues [8,9,10,11].

My implementation consists of two parts:

1. QR algorithm [12,13]: this algorithm is based on Schur factorization of a matrix from successive QR factorizations.

In [0]:
def eigenvaluesQRalgorithm(matrixA,converge=1e-8):
	if not isinstance(matrixA,np.ndarray):
		matrixA= np.array(matrixA)
  
  # QR algorithm
	n= matrixA.shape[0]
	A= matrixA
	it= 1
	Q= []
	while it < 2500:
		Q_tmp,R= QRfactorization(A)
		A= np.dot(R,Q_tmp)
		if it == 1:
			Q= Q_tmp
		else:
			Q= np.dot(Q,Q_tmp)
		it += 1 
	
	eigenval= [A[i,i] for i in range(0,n)]
	eigenvec= []
	
  # Rayleigh quotation
	for i in range(0,len(eigenval)):
		b = Q[:,i]
		mu= eigenval[i]
		matr= matrixA - mu*np.eye(n)
		
		r= np.dot(matrixA,b) - mu*b
		r= np.inner(r,r)**(0.5)
		
		iterations= 1
		while r > converge and iterations < 1000:
			tmp= linearSystemSolver(matr,b) 
			if math.isnan(np.inner(tmp,tmp)) or np.inner(tmp,tmp) == 0:
				break
			b= tmp / np.inner(tmp,tmp)
			mu= np.dot(np.dot(b.T,matrixA),b)/np.inner(b,b)
			r= np.dot(matrixA,b) - mu*b
			r= np.inner(r,r)**(0.5)
			print('r=', r)
			iterations += 1
		
		eigenval[i]= mu
		b= b / (np.inner(b,b)**(0.5))
		eigenvec.append(b)
	
	return eigenval, eigenvec


# **Results**

Present the results. If the result is an algorithm that you have described under the *Methods* section, you can present the data from verification and performance tests in this section. If the result is the output from a computational experiment this is where you present a selection of that data. 

# **Discussion**

Summarize your results and your conclusions. Were the results expected or surprising. Do your results have implications outside the particular problem investigated in this report? 

# **References**

[1] from Wikipedia, *[Direct methods](https://en.wikipedia.org/wiki/Direct_method_in_the_calculus_of_variations)*

[2] from Wikipedia, *[QR decomposition](https://en.wikipedia.org/wiki/QR_decomposition)*

[3] from Lecture Notes, *Classical Gram-Schmidt orthogonalization, chapter 5*, pg. 69

[4] from Lecture Notes, *Modified Gram-Schmidt orthogonalization, chapter 5*, pg. 70

[5] from Lecture Notes, *Triangular matrices, chapter 5*, pg. 69

[6] from Lecture Notes, *Least square problems, chapter 5*, pg. 74

[7] from Wikipedia, *[Transpose](https://en.wikipedia.org/wiki/Transpose)*

[8] from Lecture Notes, *Theorem 7, chapter 6*, pg. 80

[9] from Lecture Notes, *Theorem 8, chapter 6,* pg. 81

[10] from Lecture Notes, *Theorem 9, chapter 6,* pg. 81

[11] from Lecture Notes, *Theorem 10, chapter 6*, pg. 81

[12] from YouTube, *[QR algorithm for eigenvalues](https://www.youtube.com/watch?v=_neGVEBjLJA)*

[13] from Lecture Note, *QR algoritm, chapter 6*, pg. 82